# Method 1: Hilbert–Schmidt Test (HST)

In [ ]:
# ============================================================
# METHOD 1: Hilbert–Schmidt Test (HST) for learning Toffoli/MCX
# - Prints training progress
# - Plots training curve
# - Outputs optimized parameters
# ============================================================

import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# -------------------------
# Target gate: Multi-controlled X on last wire
# -------------------------
def target_mcx(wires):
    controls = wires[:-1]
    target = wires[-1]
    qml.MultiControlledX(control_wires=controls, wires=target, control_values="1" * len(controls))

# -------------------------
# Ansatz: Rot gates + chain CNOT entanglers (basic entangled)
# params shape: (n_layers, n_qubits, 3)
# -------------------------
def ansatz_basic(params, wires, n_layers):
    n_qubits = len(wires)
    for l in range(n_layers):
        for q in range(n_qubits):
            qml.Rot(params[l, q, 0], params[l, q, 1], params[l, q, 2], wires=wires[q])
        for q in range(n_qubits - 1):
            qml.CNOT(wires=[wires[q], wires[q + 1]])

# -------------------------
# Build HST cost function
# C_HST = 1 - |Tr(V†U)|^2 / d^2
# -------------------------
def build_hst_cost(n_qubits=3, n_layers=4, dev_name="default.qubit"):
    wires_U = list(range(n_qubits))
    wires_V = list(range(n_qubits, 2 * n_qubits))
    dev = qml.device(dev_name, wires=2 * n_qubits)

    def U():
        target_mcx(wires_U)

    def V(params):
        ansatz_basic(params, wires_V, n_layers)

    @qml.qnode(dev)
    def hs_value(params):
        # PennyLane returns |Tr(V†U)|^2 / d^2
        return qml.HilbertSchmidt(U, lambda: V(params), wires0=wires_U, wires1=wires_V)

    def cost(params):
        return 1.0 - hs_value(params)

    return cost

# -------------------------
# Training loop (with plot)
# -------------------------
def train_hst(
    n_qubits=3,
    n_layers=4,
    steps=300,
    lr=0.2,
    seed=0,
    print_every=25,
    stop_tol=1e-6
):
    np.random.seed(seed)
    cost_fn = build_hst_cost(n_qubits=n_qubits, n_layers=n_layers)

    params = np.random.normal(scale=0.1, size=(n_layers, n_qubits, 3), requires_grad=True)
    opt = qml.AdamOptimizer(lr)

    cost_history = []

    for it in range(steps):
        params, c = opt.step_and_cost(cost_fn, params)
        cost_history.append(float(c))

        if (it == 0) or ((it + 1) % print_every == 0):
            print(f"[HST] iter {it+1:4d} | cost = {c:.10f}")

        if c < stop_tol:
            print(f"[HST] Converged at iter {it+1} with cost {c:.10f}")
            break

    # Plot training curve
    plt.figure()
    plt.plot(cost_history)
    plt.xlabel("Iteration")
    plt.ylabel("Cost")
    plt.title("Method 1 (HST) Training Curve")
    plt.grid(True)
    plt.show()

    print("\nOptimized parameters (shape = layers x qubits x 3):")
    print(params)

    return params, cost_history

# -------------------------
# Run
# -------------------------
if __name__ == "__main__":
    # Example: 3-qubit Toffoli (2 controls + 1 target)
    optimized_params, history = train_hst(n_qubits=3, n_layers=4, steps=300, lr=0.2)

# Method 2 — Expected value of observable $A_n$

In [ ]:
# =========================
# METHOD 2: EXPECTED VALUE COST WITH OBSERVABLE A_n
# =========================
import pennylane as qml
from pennylane import numpy as np

# -------------------------
# Classical Toffoli mapping on bitstrings (multi-controlled X on last bit)
# -------------------------
def toffoli_map_bits(x_bits):
    """
    x_bits: list/array of 0/1, length n
    output is same as input except last bit flips if all previous bits are 1
    """
    y = list(x_bits)
    controls = y[:-1]
    if all(b == 1 for b in controls):
        y[-1] = 1 - y[-1]
    return y

# -------------------------
# Build the diagonal of A_n efficiently
# We use wire order [reg_in, reg_out] to match |in><in| ⊗ |out><out|
# -------------------------
def build_An_diagonal(n):
    """
    Returns diag entries of A_n as a vector of length 2^(2n).
    A_n = I - 2G, where G has 1 on basis states |in>|out> that match Toffoli truth table.
    So diag(A_n) is -1 on "correct pairs" and +1 otherwise.
    """
    dim = 2 ** (2 * n)
    diag = np.ones(dim, dtype=float)

    # Enumerate all inputs x (0..2^n - 1)
    for xin in range(2 ** n):
        x_bits = [(xin >> (n - 1 - b)) & 1 for b in range(n)]
        y_bits = toffoli_map_bits(x_bits)

        # Convert y_bits to integer
        yout = 0
        for b in range(n):
            yout = (yout << 1) | y_bits[b]

        # Basis index for |in>|out> in order (in as most-significant block)
        idx = xin * (2 ** n) + yout

        # On correct pairs, diag(A_n) = 1 - 2*1 = -1
        diag[idx] = -1.0

    return diag

# -------------------------
# Convert diagonal into full Hermitian matrix (still easy for n<=5)
# -------------------------
def diagonal_to_matrix(diag):
    return np.diag(diag)

# -------------------------
# Ansatz (same style as Method 1)
# -------------------------
def ansatz_basic(params, wires, n_layers):
    n_wires = len(wires)
    for l in range(n_layers):
        for w in range(n_wires):
            qml.Rot(params[l, w, 0], params[l, w, 1], params[l, w, 2], wires=wires[w])
        for w in range(n_wires - 1):
            qml.CNOT(wires=[wires[w], wires[w + 1]])

# -------------------------
# Build expected-value cost
# -------------------------
def build_expected_value_cost(n_qubits=3, n_layers=4, dev_name="default.qubit"):
    """
    We use 2*n wires total:
      reg_out = wires [0..n-1]   (this register will go through ansatz)
      reg_in  = wires [n..2n-1]  (this will store the copied input)

    Observable A_n is defined on (reg_in ⊗ reg_out), so we pass wires in that order.
    """
    reg_out = list(range(n_qubits))
    reg_in = list(range(n_qubits, 2 * n_qubits))
    dev = qml.device(dev_name, wires=2 * n_qubits)

    # Build A_n matrix
    diag = build_An_diagonal(n_qubits)
    An = diagonal_to_matrix(diag)

    wires_for_observable = reg_in + reg_out  # order: input-register first, output-register second
    obs = qml.Hermitian(An, wires=wires_for_observable)

    @qml.qnode(dev)
    def expval_circuit(params):
        # Start |0...0> on both registers

        # Create uniform superposition of all inputs on reg_out (the "input" superposition |phi>)
        for w in reg_out:
            qml.Hadamard(wires=w)

        # Copy computational-basis inputs from reg_out to reg_in (CNOTs)
        for a, b in zip(reg_out, reg_in):
            qml.CNOT(wires=[a, b])

        # Apply the ansatz U(theta) to reg_out, producing the output state on reg_out
        ansatz_basic(params, reg_out, n_layers)

        # Measure expected value of A_n
        return qml.expval(obs)

    def cost(params):
        # expval is -1 when all pairs match; +1 otherwise
        # Map it to a nonnegative objective: 0 is best
        return 0.5 * (expval_circuit(params) + 1.0)

    return cost

# -------------------------
# Train
# -------------------------
def train_expected_value(n_qubits=3, n_layers=4, steps=300, lr=0.2, seed=0):
    np.random.seed(seed)
    cost = build_expected_value_cost(n_qubits=n_qubits, n_layers=n_layers)

    params = np.random.normal(scale=0.1, size=(n_layers, n_qubits, 3), requires_grad=True)
    opt = qml.AdamOptimizer(lr)

    for it in range(steps):
        params, c = opt.step_and_cost(cost, params)
        if (it + 1) % 25 == 0 or it == 0:
            print(f"[ExpVal] iter {it+1:4d}  cost = {c:.8f}")
        if c < 1e-6:
            print("Converged.")
            break

    return params

if __name__ == "__main__":
    learned_params = train_expected_value(n_qubits=3, n_layers=4, steps=300, lr=0.2)